In [ ]:
from IPython.core.display import HTML
display(HTML('<style>.container { width:95% !important; } </style>'))

%load_ext autoreload
%autoreload 2

## Policy Gradient Algorithms


The main goal of this TD is helping you understand Policy Gradient (PG) algorithms like REINFORCE, TRPO and PPO. 
You will be provided with some boilerplate code to skip the boring parts, and will need to implement the actual update logic of the algorithms.

The following code is inspired from [CleanRL](https://github.com/vwxyzjn/cleanrl), which is a good general reference for implementation of RL algorithms. I suggest you do not look at its code before getting something to work at least somewhat well -- but don't hesitate to look at these and other implementations later, to see how the tiny details might differ (sometimes in an impactful way!).

In particular, hyperparameter tuning is a crucial part of practical deep RL as small tweaks in values or implementation can result in dramatic performance changes. Values provided in this notebook should not be *too* bad, but feel free to explore more!

## Environments

In this exercise we will use the [gym](https://github.com/openai/gym) library for the environment. Later, we will try our algorithms on a Unity MLAgents environment.

In [ ]:
# Imports
import sys
import abc
from tqdm.notebook import tqdm, trange
import numpy as np
import time

import gym
from gym.wrappers import RecordVideo

import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions.categorical import Categorical

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
# Detect if CUDA acceleration is available
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

In [ ]:
print(f"python --version = {sys.version}")
print(f"torch.__version__ = {torch.__version__}")
print(f"np.__version__ = {np.__version__}")
print(f"gym.__version__ = {gym.__version__}")

## Torch 101

>"The torch package contains data structures for multi-dimensional tensors and defines mathematical operations over these tensors. Additionally, it provides many utilities for efficient serializing of Tensors and arbitrary types, and other useful utilities. 
[...] provides classes and functions implementing automatic differentiation of arbitrary scalar valued functions." 
[PyTorch](https://pytorch.org/docs/stable/index.html)


### Variable types

In [ ]:
# Very similar syntax to numpy.
zero_torch = torch.zeros((3, 2))

print('zero_torch is of type {:s}'.format(str(type(zero_torch))))

# Torch -> Numpy: simply call the numpy() method.
zero_np = np.zeros((3, 2))
assert (zero_torch.numpy() == zero_np).all()

# Numpy -> Torch: simply call the corresponding function on the np.array.
zero_torch_float = torch.FloatTensor(zero_np)
print('\nFloat:\n', zero_torch_float)
zero_torch_int = torch.LongTensor(zero_np)
print('Int:\n', zero_torch_int)
zero_torch_bool = torch.BoolTensor(zero_np)
print('Bool:\n', zero_torch_bool)

# Reshape
print('\nView new shape...', zero_torch.view(1, 6))
# Note that print(zero_torch.reshape(1, 6)) would work too.
# The difference is in how memory is handled (view imposes contiguity).

# Algebra
a = torch.randn((3, 2))
b = torch.randn((3, 2))
print('\nAlgebraic operations are overloaded:\n', a, '\n+\n', b, '\n=\n', a+b )

# More generally, torch shares the syntax of many attributes and functions with Numpy.

### Gradient management

In [ ]:
# torch.Tensor is a similar yet more complicated data structure than np.array.
# It is basically a static array of number but may also contain an overlay to 
# handle automatic differentiation (i.e keeping track of the gradient and which 
# tensors depend on which).
# To access the static array embedded in a tensor, simply call the detach() method
print(zero_torch.detach())

# When inside a function performing automatic differentiation (basically when training 
# a neural network), never use detach() otherwise meta information regarding gradients
# will be lost, effectively freezing the variable and preventing backprop for it. 
# However when returning the result of training, do use detach() to save memory 
# (the naked tensor data uses much less memory than the full-blown tensor with gradient
# management, and is much less prone to mistake such as bad copy and memory leak).

# We will solve theta * x = y in theta for x=1 and y=2
x = torch.ones(1)
y = 2 * torch.ones(1)

# Actually by default torch does not add the gradient management overlay
# when declaring tensors like this. To force it, add requires_grad=True.
theta = torch.randn(1, requires_grad=True)

# Optimisation routine
# (Adam is a sophisticated variant of SGD, with adaptive step).
optimizer = optim.Adam(params=[theta], lr=0.1)

# Loss function
print('Initial guess:', theta.detach())

for _ in range(100):
    # By default, torch accumulates gradients in memory.
    # To obtain the desired gradient descent beahviour,
    # just clean the cached gradients using the following line:
    optimizer.zero_grad()
    
    # Quadratic loss (* and ** are overloaded so that torch
    # knows how to differentiate them)
    loss = (y - theta * x) ** 2
    
    # Apply the chain rule to automatically compute gradients
    # for all relevant tensors.
    loss.backward()
    
    # Run one step of optimisation routine.
    optimizer.step()
    
print('Final estimate:', theta.detach())
print('The final estimate should be close to', y)

## CartPole again

In [ ]:
# Create an environment
env = gym.make('CartPole-v0')

In [ ]:
obs = env.reset()
print(obs)

In [ ]:
obs, reward, done, info = env.step(0)
print(f"New observation: {obs}")
print(f"Reward: {reward}")
print(f"Episode finished: {done}")
print(f"Extra info: {info}")

In [ ]:
# Some extra info about the environment
print(f"Observation space: {env.observation_space}")
print(f"Observation space shape: {env.observation_space.shape}")
print(f"Action space: {env.action_space}")

## Playing a random episode of CartPole

Showing/recording a video with a jupyter notebook is always tricky, the below works with Chrome but might fail or behave weirdly in a different browser.

In [ ]:
done = False
state = env.reset()
while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)
    env.render(mode='rgb_array')
env.close()

## Agent 
Let's implement the Agent class. It represents the policy (actor) $\pi_\theta\colon \mathcal{S} \to \mathcal{A}$, as well as the value estimator (critic) $V_{\theta'}\colon \mathcal{S} \to \mathbb{R}$, where $\theta, \theta'$ are parameters (typically the weights and biases of neural network approximators).

Question: implement a dense neural network (MLP) in the class Net.

In [ ]:
class Net(nn.Module):
    """
    Basic neural net.
    """
    def __init__(self, n_input, hidden_size, n_output):
        super(Net, self).__init__()
        self.net = # TODO

    def forward(self, x):
        return self.net(x)


class Agent(nn.Module):
    def __init__(self, env: gym.Env):
        super().__init__()
        # Hint: use the fact that the environment is an argument here, to deduce the input and output shapes of the network
        
        # Note - you can structure your code differently. This is just one of the options.
        self.critic = # TODO
        self.actor = # TODO

    def get_value(self, x: Tensor) -> Tensor:
        """Return an estimation of the learned value function of state x.
        """
        return # TODO

    def get_action(self, x: Tensor, action=None) -> ...:
        """
        If action is not None, return this action, otherwise return a 
        random action selected according to the learned policy.
        
        Hint: you might want to return some more properties of the 
        action distribution than just the selected action...
      """
        # TODO
        if action is None:
            action = probs.sample()
        return action, ...

## Training

Now we're getting into the meat of the algorithm - the training loop.
First let's do some bookkeeping.

In [ ]:
# Gym environment name
# ENV_NAME = 'CartPole-v0'  # shorter episodes
ENV_NAME = 'CartPole-v1'

# Discount factor
GAMMA = 0.99

# Number of epoch to train neural networks
N_EPOCH = 10

# Size of minibatch (sample one minibatch per epoch)
MB_SIZE = 32

# How many steps (environment iterations) per training iteration?
N_STEPS = 500
 
# How many agent updates
N_UPDATES = 50

In [ ]:
class TrainMgr(abc.ABC):
    """Training Manager.
    Implements the episodic training of an algorithm
    specified in a child class.
    """
    def __init__(self, env_name, device, gamma=0.99):
        self.env_name = env_name
        self.device = device
        self.gamma = gamma
        self.reset()

    def reset(self):
        self.env = gym.make(self.env_name)
        self.agent = Agent(self.env)
        self.actor_optimizer = optim.Adam(self.agent.actor.parameters(), lr=3e-4, eps=1e-5)
        self.critic_optimizer = optim.Adam(self.agent.critic.parameters(), lr=3e-4, eps=1e-5)
        
        self.mean_total_rewards = []
        self.q_10_total_rewards = []
        self.q_90_total_rewards = []

    def generate_rollout(self, num_steps, start_obs, start_done):
        """Play during num_steps, reset episode if the agent reaches the end.
        """
        # Initialize run
        next_obs = start_obs
        next_done = start_done

        # Initialize some storage buffers
        obs = torch.zeros((num_steps,) + self.env.observation_space.shape).to(self.device)
        actions = torch.zeros((num_steps,) + self.env.action_space.shape).to(self.device)
        rewards = torch.zeros((num_steps,)).to(self.device)
        dones = torch.zeros((num_steps,)).to(self.device)
        # TODO: Something else?
        
        for step in range(0, num_steps):
            obs[step] = next_obs
            dones[step] = next_done

            # Get the action and everything else necessary from the agent
            # action, ... = self.agent.get_action(next_obs)
            
            # Record some data
            actions[step] = action

            # Take a step in the environment
            next_obs, reward, done, info = self.env.step(action.cpu().numpy())
            # Record the reward
            rewards[step] = torch.tensor(reward).to(self.device)

            if done:
                next_obs = self.env.reset()

            next_obs, next_done = torch.Tensor(next_obs).to(self.device), torch.Tensor([done]).to(self.device)

        return obs, actions, rewards, dones #, something else?

    def full_episode_rewards(self, rewards, dones):
        """Slice and dice the rollout to get total rewards per episode.
        """
        ep_rewards = []
        current_ret = 0
        for r, d in zip(rewards, dones):
            if d:
                ep_rewards.append(current_ret)
                current_ret = 0
            current_ret += r
        if len(ep_rewards) == 0:
            ep_rewards.append(current_ret)
        return ep_rewards
    
    def render(self):
        done = False
        next_obs = torch.Tensor(self.env.reset()).to(self.device)
        while not done:
            with torch.no_grad():
                action, _, _ = self.agent.get_action(next_obs)
            state, reward, done, info = self.env.step(action.cpu().numpy())
            self.env.render(mode='rgb_array')
        self.env.close()        

    @abc.abstractmethod
    def fit(self):
        pass
    

def plot_xp(algs):
    """Algs is a list of TrainMgr object (or children of it).
    """
    colors = plt.cm.rainbow(np.linspace(0, 1, len(algs)))
    
    with sns.axes_style('darkgrid'):
        fig, ax = plt.subplots(figsize=(7, 5), nrows=1, ncols=1)
        
        for i, alg in enumerate(algs):
            ax.plot(range(1, N_UPDATES + 1), alg.mean_total_rewards, color=colors[i], label=alg.name)
            ax.fill_between(range(1, N_UPDATES + 1), alg.q_10_total_rewards, alg.q_90_total_rewards, color=colors[i], alpha=0.2)

        ax.set_xlabel('Number of updates')
        ax.set_ylabel('Returns')
        ax.legend(loc='lower right', prop={'size': 12})

        plt.tight_layout()
        plt.show()

## REINFORCE

Let $\tau = (s_1, a_1, r_1, \dots, s_{H+1})$ a rollout of the MDP following a policy $\pi_\theta$ and $R_\tau=\sum_{t=1}^H r_t$ the total reward, or *return* of the rollout $\tau$.

The goal of an agent is to maximize 
$$J(\theta) = \mathbb{E}[R_\tau],$$
where the expectation is over the randomness of the MDP (transitions, rewards) and the stochastic policy $\pi_\theta$.

In order to find the parameter $\theta$ (i.e the policy) that maximizes the expected return, it is natural to compute the gradient $\nabla_\theta J$.
The Policy Gradient Theorem states that
$$ \nabla_\theta J(\theta) = \mathbb{E}[R_\tau \nabla_\theta \log \pi_\theta].$$

REINFORCE is a simple algorithm that implements gradient *ascent* on an estimation of $J(\theta)$ sampled from multiple episodes. 

In [ ]:
class ReinforceMgr(TrainMgr):
    def __init__(self, env_name, device, gamma=0.99):
        self.name = 'REINFORCE'
        super().__init__(env_name, device, gamma)
    
    def fit(self, n_updates, n_steps, n_epoch, mb_size):
        start_time = time.time()
    
        self.mean_total_rewards = np.zeros(n_updates)
        self.q_10_total_rewards = np.zeros(n_updates)
        self.q_90_total_rewards = np.zeros(n_updates)

        for update in trange(n_updates):
            start_obs = torch.Tensor(self.env.reset()).to(self.device)
            start_done = torch.zeros((1,)).to(self.device)

            # Generate rollout
            obs, actions, rewards, dones = self.generate_rollout(n_steps, start_obs, start_done)

            # Compute average total rewards
            ep_rewards = self.full_episode_rewards(rewards, dones)
            self.mean_total_rewards[update] = np.mean(ep_rewards)
            self.q_10_total_rewards[update] = np.quantile(ep_rewards, 0.1)
            self.q_90_total_rewards[update] = np.quantile(ep_rewards, 0.9)

            # Now you need to compute the returns.
            # Remember that the return is defined as the discounted sum of rewards from that step onwards.
            with torch.no_grad():
                returns = # TODO

            # Flatten the batch
            b_obs = obs.reshape((-1,) + self.env.observation_space.shape)
            b_actions = actions.reshape((-1,) + self.env.action_space.shape)
            b_returns = returns.reshape(-1)

            # Optimizing the policy network

            # Your time to shine! Compute the necessary gradients, and apply them to the network.
            # Feel free to modify the code however you see fit.
            b_inds = np.arange(n_steps)

            for epoch in range(n_epoch):
                np.random.shuffle(b_inds)
                for start in range(0, n_steps, mb_size):
                    end = start + mb_size

                    # Minibatch
                    mb_inds = b_inds[start:end]
                    mb_returns = b_returns[mb_inds]

                    # TODO

        self.train_time = time.time() - start_time

In [ ]:
reinforce = ReinforceMgr(ENV_NAME, DEVICE)
reinforce.fit(N_UPDATES, N_STEPS, N_EPOCH, MB_SIZE)
plot_xp([reinforce])

In [ ]:
reinforce.render()

## REINFORCE with baseline

A fruitful idea to speed up a stochastic algorithm that estimates an expectation is *variance reduction*.
Indeed, if $R$ is a random variable and $R^1, \dots, R^n$ i.i.d samples of $R$, then (under mild assumptions) $\frac{1}{n}\sum_{i=1}^nR^i \rightarrow \mathbb{E}[R]$ at rate $\sqrt{\frac{Var(R)}{n}}$. 

A standard variance reduction technique is to use a *baseline* variable $B$ such that $\mathbb{E}[B]=0$ and estimate $\mathbb{E}[R-B]$.

1) When is the variance of $R-B$ smaller than the variance of $R$?

In the context of REINFORCE, $R$ is the return of an episode, and we estimate $\mathbb{E}[R\log\nabla_\theta \pi_\theta]$.

2) Explain why $B$ independent of the policy $\pi_\theta$ is a suitable choice of baseline. Suggest one and implement the corresponding REINFORCE with baseline.

Note: one option is to have $B$ be the outcome of *another* neural network trained to estimate a certain quantity...

In [ ]:
class ReinforceBaselineMgr(TrainMgr):
    def __init__(self, env_name, device, gamma=0.99):
        self.name = 'REINFORCE_baseline'
        super().__init__(env_name, device, gamma)
    
    
    def fit(self, n_updates, n_steps, n_epoch, mb_size):
        # TODO

In [ ]:
reinforce_b = ReinforceBaselineMgr(ENV_NAME, DEVICE)
reinforce_b.fit(N_UPDATES, N_STEPS, N_EPOCH, MB_SIZE)
plot_xp([reinforce, reinforce_b])

## PPO

TRPO/PPO are algorithms that implement the policy gradient ascent using the *natural gradient* framework.

1) Recall what natural gradient is.

TRPO solves a constrained optimization problem 
$$
\max J_{\pi_k}(\theta)\quad \text{s.t}\quad \mathbb{E}[\text{KL}(\pi_\theta \lVert \pi_k)] \leq \delta,
$$
while PPO solves the penalized optimization problem
$$
\max J_{\pi_k}(\theta) -\lambda_k \mathbb{E}[\text{KL}(\pi_\theta \lVert \pi_k)].
$$

In practice, PPO is implemented in a different, simpler manner.
Consider the objective function at step $k$, that is policy $\pi_k=\pi_{\theta_k}$ has been learned and used in the previous rollout and we want to train $\pi_{\theta}$:
$$
J_{\pi_k}(\theta) = \mathbb{E}_{\pi_k}[\frac{\pi_\theta(s, a)}{\pi_k(s,a)} R]
$$
We define the clipped surrogate objective as:
$$
J^{CLIP}_{\pi_k}(\theta) = \mathbb{E}_{\pi_k}[\min(\frac{\pi_\theta(s, a)}{\pi_k(s,a)} R, \text{clip}(\frac{\pi_\theta(s, a)}{\pi_k(s,a)}, 1-\epsilon, 1+\epsilon) R)].
$$

2) How do you interpret this surrogate objective? How does it relate to the form of PPO above? What is the practical advantage of this approach? Does it remind you of some other technique in deep learning?

In practice, $\epsilon=0.1$ or $\epsilon=0.2$ are good default.

In [ ]:
class PPOMgr(TrainMgr):
    def __init__(self, env_name, device, gamma=0.99, clip_coeff=0.2):
        self.name = 'PPO'
        self.clip_coeff = clip_coeff
        super().__init__(env_name, device, gamma)

    def fit(self, n_updates, n_steps, n_epoch, mb_size):
        # TODO

In [ ]:
ppo = PPOMgr(ENV_NAME, DEVICE)
ppo.fit(N_UPDATES, N_STEPS, N_EPOCH, MB_SIZE)
plot_xp([reinforce, reinforce_b, ppo])

## Additional Question

Many modern RL algorithms implement exploration in policy gradient as entropy regularization. The entropy of a distribution $\pi$ is $H(\pi)=-\mathbb{E}_{X\sim\pi}[\log \pi(X)]$. High entropy corresponds to more diffuse distributions, i.e there is more chance to explore different states.

1) If you are not familiar with entropy, compute the $H(\pi)$ where $\pi\mathcal{\mu, \sigma^2}$ is a Gaussian policy.

2) Implement entropy regularization in the above algorithms.
Hint: entropy can be returned by agent.get_action and then used during training as an extra loss term.

# Bonus: Unity ML-Agents

In [ ]:
from mlagents_envs.environment import UnityEnvironment
from gym_unity.envs import UnityToGymWrapper
import os


def make_unity_environment():
    # Create an environment
    if os.name == 'posix':
        unity_env = UnityEnvironment('./3DBallSingleAgent/3DBallSingleAgent_unix/3DBallSingleAgent.x86_64')
    elif os.name == 'nt':
        unity_env = UnityEnvironment('./3DBallSingleAgent/3DBallSingleAgent_windows')
    else:
        raise ValueError('Sorry, your OS is not supported :(')
    return UnityToGymWrapper(unity_env, 0)

In [ ]:
env = make_unity_environment()
# Do some testing
obs = env.reset()
print(obs)

action = env.action_space.sample()
obs, reward, done, info = env.step(action)
env.close()

print(f"New observation: {obs}")
print(f"Reward: {reward}")
print(f"Episode finished: {done}")
print(f"Extra info: {info}")

print(f"Observation space: {env.observation_space}")
print(f"Observation space shape: {env.observation_space.shape}")
print(f"Action space: {env.action_space}")
print(f"Action space shape: {env.action_space.shape}")

## Agent

The 3DBall environment has a 2D *continuous* action space. This is new compared to CartPole as in the latter the agent could only stir left or right ($\mathcal{A}=\{0, 1\}$).

1) Could one use DQN on this problem? Why or why not?

2) Propose one or several ways to modify the above algorithms in order to handle this 2D action space. Implement them.

Disclaimer: training may be slow, do not worry if it does not converge. The goal of this part is to show you that it is possible to interface Unity with gym and tackle more complex environments than CartPole.

In [ ]:
from torch.distributions.beta import Beta

class Net(nn.Module):
    """
    Basic neural net.
    """
    def __init__(self, n_input, hidden_size, n_output):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(n_input, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, n_output)
        )

    def forward(self, x):
        return self.net(x)


from torch.distributions.multivariate_normal import MultivariateNormal

class Agent(nn.Module):
    def __init__(self, env: gym.Env):
        super().__init__()
        # Hint: use the fact that the environment is an argument here, to deduce the input and output shapes of the network
        
        # Note - you can structure your code differently. This is just one of the options.
        self.critic = # TODO
        self.actor = # TODO

    def get_value(self, x: Tensor) -> Tensor:
        """Return an estimation of the learned value function of state x.
        """
        return # TODO

    def get_action(self, x: Tensor, action=None) -> ...:
        """
        If action is not None, return this action, otherwise return a 
        random action selected according to the learned policy.
        
        Hint: you might want to return some more properties of the 
        action distribution than just the selected action...
      """
        # TODO
        if action is None:
            action = probs.sample()
        return action, ...

In [ ]:
# Discount factor
GAMMA = 0.99

# Number of epoch to train neural networks
N_EPOCH = 10

# Size of minibatch (sample one minibatch per epoch)
MB_SIZE = 32

# How many steps (environment iterations) per training iteration?
N_STEPS = 100
 
# How many agent updates
N_UPDATES = 50

In [ ]:
class PPOM3DBallMgr():
    def __init__(self, env_name, device, gamma=0.99, clip_coeff=0.2):
        self.name = 'PPO3DBall'
        self.device = device
        self.gamma = gamma
        self.clip_coeff = clip_coeff
        self.reset()
        
    def reset(self):
        self.env = make_unity_environment()
        self.agent = Agent(self.env)
        self.actor_optimizer = optim.Adam(self.agent.actor.parameters(), lr=3e-4, eps=1e-5)
        self.critic_optimizer = optim.Adam(self.agent.critic.parameters(), lr=3e-4, eps=1e-5)
        
        self.mean_total_rewards = []
        self.q_10_total_rewards = []
        self.q_90_total_rewards = []


In [ ]:
ppo_unity = PPOM3DBallMgr(ENV_NAME, DEVICE)
ppo_unity.fit(N_UPDATES, N_STEPS, N_EPOCH, MB_SIZE)

In [ ]:
plot_xp([ppo_unity])

In [ ]:
ppo_unity.env.close()